In [1]:
import pandas as pd
import numpy as np


# Load Raw Data

In [7]:
data = pd.read_excel("../data/raw/ethiopia_fi_unified_data.xlsx")
codes = pd.read_excel("../data/raw/reference_codes.xlsx")

print(data.shape)
data.head()


(43, 34)


,record_id,record_type,category,pillar,indicator,indicator_code,indicator_direction,value_numeric,value_text,value_type,...,impact_direction,impact_magnitude,impact_estimate,lag_months,evidence_basis,comparable_country,collected_by,collection_date,original_text,notes
0,REC_0001,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,22.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Baseline year,NaN
1,REC_0002,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,35.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,NaN,NaN
2,REC_0003,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,46.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,NaN,NaN
3,REC_0004,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,56.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Gender disaggregated,NaN
4,REC_0005,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,36.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Gender disaggregated,NaN


# Schema & Record Types

In [8]:
data.info()
data['record_type'].value_counts()


<class 'pandas.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 34 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   record_id            43 non-null     str           
 1   record_type          43 non-null     str           
 2   category             10 non-null     str           
 3   pillar               33 non-null     str           
 4   indicator            43 non-null     str           
 5   indicator_code       43 non-null     str           
 6   indicator_direction  33 non-null     str           
 7   value_numeric        33 non-null     float64       
 8   value_text           10 non-null     str           
 9   value_type           43 non-null     str           
 10  unit                 33 non-null     str           
 11  observation_date     43 non-null     datetime64[us]
 12  period_start         10 non-null     datetime64[us]
 13  period_end           10 non-null     datetime64[

record_type
observation    30
event          10
target          3
Name: count, dtype: int64

# Indicator Coverage

In [9]:
data['indicator_code'].value_counts()


indicator_code
ACC_OWNERSHIP         7
ACC_FAYDA             4
ACC_MM_ACCOUNT        2
ACC_4G_COV            2
USG_P2P_COUNT         2
GEN_GAP_ACC           2
GEN_MM_SHARE          2
ACC_MOBILE_PEN        1
USG_P2P_VALUE         1
USG_ATM_COUNT         1
USG_ATM_VALUE         1
USG_CROSSOVER         1
USG_TELEBIRR_USERS    1
USG_TELEBIRR_VALUE    1
USG_MPESA_USERS       1
USG_MPESA_ACTIVE      1
USG_ACTIVE_RATE       1
AFF_DATA_INCOME       1
GEN_GAP_MOBILE        1
EVT_TELEBIRR          1
EVT_SAFARICOM         1
EVT_MPESA             1
EVT_FAYDA             1
EVT_FX_REFORM         1
EVT_CROSSOVER         1
EVT_MPESA_INTEROP     1
EVT_ETHIOPAY          1
EVT_NFIS2             1
EVT_SAFCOM_PRICE      1
Name: count, dtype: int64

# Time Coverage

In [10]:
data['observation_date'] = pd.to_datetime(
    data['observation_date'], errors='coerce'
)

data.groupby('indicator_code')['observation_date'].agg(['min','max'])


,min,max
indicator_code,,
ACC_4G_COV,2023-06-30,2025-06-30
ACC_FAYDA,2024-08-15,2028-12-31
ACC_MM_ACCOUNT,2021-12-31,2024-11-29
ACC_MOBILE_PEN,2025-12-31,2025-12-31
ACC_OWNERSHIP,2014-12-31,2025-12-31
AFF_DATA_INCOME,2024-12-31,2024-12-31
EVT_CROSSOVER,2024-10-01,2024-10-01
EVT_ETHIOPAY,2025-12-18,2025-12-18
EVT_FAYDA,2024-01-01,2024-01-01


# Add New Observation

In [11]:
new_observation = {
    "record_type": "observation",
    "pillar": "usage",
    "indicator": "Smartphone Penetration",
    "indicator_code": "USG_SMARTPHONE_PEN",
    "value_numeric": 44.0,
    "observation_date": "2023-01-01",
    "source_name": "GSMA",
    "source_url": "https://www.gsma.com",
    "confidence": "medium",
    "notes": "Smartphone penetration enables digital payment usage"
}

data = pd.concat([data, pd.DataFrame([new_observation])], ignore_index=True)


# Add New Event

In [12]:
new_event = {
    "record_type": "event",
    "event_name": "Fayda Digital ID National Rollout",
    "category": "policy",
    "event_date": "2023-01-01",
    "pillar": "",
    "source_name": "Government of Ethiopia",
    "source_url": "https://id.gov.et",
    "confidence": "medium",
    "notes": "Digital ID simplifies KYC requirements"
}

data = pd.concat([data, pd.DataFrame([new_event])], ignore_index=True)


# Add Impact Link

In [13]:
new_impact = {
    "record_type": "impact_link",
    "parent_id": "Fayda Digital ID National Rollout",
    "pillar": "access",
    "related_indicator": "ACC_OWNERSHIP",
    "impact_direction": "positive",
    "impact_magnitude": "medium",
    "lag_months": 12,
    "evidence_basis": "Comparable effects observed in India Aadhaar rollout"
}

data = pd.concat([data, pd.DataFrame([new_impact])], ignore_index=True)


# Save Enriched Dataset

In [14]:
data.to_csv("../data/processed/ethiopia_fi_enriched.csv", index=False)
